In [ ]:
!pip install qubovert > h

**Exercise 1**

The famous $N$ queens problem consists of placing $N$ queens on a $N \times N$ chessboard in such a way that none of them attacks any other. Model in QUBO the problem using `qubovert` and solve it for the case $N=8$.

**Solution**

In [ ]:
import qubovert

size = 8
lagrange = 1

# We create the variables of our model
Q = qubovert.QUBO()
for i in range(size):
    for j in range(size):
        Q.create_var(f"x_{i}_{j}")

# We add the first block of the objective function
for i in range(size):
    for j in range(size):
        Q[(f"x_{i}_{j}",)] = -1

# We include the final restrictions
for i1 in range(size):
    for i2 in range(size):
        for i3 in range(size):
            for i4 in range(size):
                if i1 == i3 or i2 == i4 or i1 - i3 == i2 - i4 or i1 - i3 == i4 - i2:
                    if not (i1 == i3 and i2 == i4):
                        Q[(f"x_{i1}_{i2}", f"x_{i3}_{i4}")] = lagrange

Let's solve using the `qubovert` annealer and plot the results.

In [ ]:
anneal_res = qubovert.sim.anneal_pubo(Q, num_anneals= 100)
for i in range(size):
    for j in range(size):
        if anneal_res.best.state[f"x_{i}_{j}"] == 0:
            print("O", end = " ")
        else:
            print("X", end = " ")
    print()

O O O O O O X O 
O X O O O O O O 
O O O O O X O O 
O O X O O O O O 
X O O O O O O O 
O O O X O O O O 
O O O O O O O X 
O O O O X O O O 


/usr/local/lib/python3.11/dist-packages/qubovert/sim/_anneal.py:603: QUBOVertWarning: The input problem has degree <= 2; consider using the ``qubovert.sim.anneal_qubo`` or ``qubovert.sim.anneal_quso`` functions, which are significantly faster than this function because they take advantage of the low degree.
  return anneal_puso(


**Exercise 2**

The knapsack problem states that we have a number of objects with an associated value and weight (in kg).

```python
value = [1,4,2,2,5,2]
weight = [1,3,3,3,2,2]
```

You are given a maximum $L$ weight that you can support. Say in this case:

```python
L = 7
```

Your goal will be to create a QUBO problem that determines which objects we should take to maximize the value but not exceed the weight.
Use `qubovert` to find the solution.

In [ ]:
import qubovert

value = [1,4,2,2,5,2]
weight = [1,3,3,3,2,2]
L = 7

# create QUBO model
Q = qubovert.QUBO()
for i in range(len(value)):
    Q.create_var(f"x_{i}")

# maximize value
for i in range(len(value)):
    Q[(f"x_{i}",)] = -value[i]

# constraint: weight <= L
lagrange = 10
for i in range(len(weight)):
    for j in range(i+1, len(weight)):
        Q[(f"x_{i}",f"x_{j}")] = lagrange * weight[i]*weight[j]

for i in range(len(weight)):
    Q[(f"x_{i}",)] += lagrange * weight[i]**2 - lagrange * L * weight[i]

# solve the model
anneal_res = qubovert.sim.anneal_pubo(Q, num_anneals=100)

# extract the solution
solution = {}
total_value = 0
total_weight = 0
for i in range(len(value)):
  solution[i] = anneal_res.best.state[f"x_{i}"]
  if solution[i] == 1:
    total_value += value[i]
    total_weight += weight[i]

print("Solution:", solution)
print("Total Value", total_value)
print("Total Weight", total_weight)


Solution: {0: 1, 1: 0, 2: 0, 3: 0, 4: 1, 5: 1}
Total Value 8
Total Weight 5
